In [1]:
import vgg16
from keras.utils.data_utils import get_file


Using Theano backend.
/home/hyz/.virtualenvs/myenv/local/lib/python2.7/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7001 on context None
Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:01:00.0)


In [2]:
v = vgg16.Vgg16()
fname = 'vgg16.h5'
FILE_PATH = 'http://files.fast.ai/models/'
v.model.load_weights(get_file(fname, FILE_PATH+fname, cache_subdir='models'))

/home/hyz/.virtualenvs/myenv/local/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/hyz/.virtualenvs/myenv/local/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/hyz/.virtualenvs/myenv/local/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/hyz/.virtualenvs/myenv/local/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)


In [3]:
v.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [4]:
import numpy as np
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D 
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


In [5]:
img_height = 224
img_width = img_height
batch_size=8

In [6]:
v.model.pop()
for layer in v.model.layers:
    layer.trainable=False
v.model.add(Dense(10, activation='softmax'))
#v.model.layers[-2].trainable= True
#v.model.layers[-3].trainable= True

v.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [7]:
train_datagen = ImageDataGenerator(
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                               )

test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (img_height, img_width),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('val',
                                            target_size = (img_height, img_width),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')
filepath="weights-improvement-3--{epoch:02d}-{categorical_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

v.model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy', metrics=['categorical_accuracy'])



Found 14607 images belonging to 10 classes.
Found 7823 images belonging to 10 classes.


In [8]:
v.model.fit_generator(training_set,
                         steps_per_epoch = 14000/batch_size,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = 7000/batch_size, callbacks=callbacks_list)

Epoch 1/5
1750/1750 [==============================] - 249s - loss: 2.3975 - categorical_accuracy: 0.4371 - val_loss: 0.6568 - val_categorical_accuracy: 0.7873
Epoch 2/5
1750/1750 [==============================] - 251s - loss: 2.1171 - categorical_accuracy: 0.5398 - val_loss: 0.7670 - val_categorical_accuracy: 0.7761
Epoch 3/5
1750/1750 [==============================] - 252s - loss: 2.1960 - categorical_accuracy: 0.5658 - val_loss: 0.6569 - val_categorical_accuracy: 0.7984
Epoch 4/5
1750/1750 [==============================] - 252s - loss: 2.1674 - categorical_accuracy: 0.5926 - val_loss: 0.5179 - val_categorical_accuracy: 0.8523
Epoch 5/5
1750/1750 [==============================] - 253s - loss: 2.2663 - categorical_accuracy: 0.5972 - val_loss: 0.6253 - val_categorical_accuracy: 0.8287


In [9]:
import csv 
submit_datagen = ImageDataGenerator() #'sub/test'
submit_set = submit_datagen.flow_from_directory('sub/test', target_size= (img_width, img_height), batch_size=1, class_mode=None)

with open('submission2.csv', 'w') as csvfile:
  csvw = csv.writer(csvfile)
  ct = 0
  for i in submit_set:
    idx = (submit_set.batch_index -1 ) * submit_set.batch_size
    ct = ct + 1
    if ct > 79726:
      break
    #if idx < 0:
    #  break
    if idx % 100 == 0:
      print idx
    fnames = submit_set.filenames[idx: idx+ submit_set.batch_size]
    res = np.round(v.model.predict(i), decimals=0)
    #print fnames, res
    #break

    for fn, sc in zip(fnames,res):
      #print fn, sc
      csvw.writerow([fn.split('/')[1]] + sc.tolist())

Found 79726 images belonging to 1 classes.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
1

In [10]:
t = submit_set.next()


In [12]:
v.model.predict(t)

array([[  9.82805550e-01,   2.67333348e-06,   1.73947177e-04,
          2.41549856e-08,   7.74689106e-05,   1.86598581e-09,
          7.13222520e-03,   1.37951295e-03,   1.12423249e-05,
          8.41738656e-03]], dtype=float32)

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
t[0].shape
#plt.imshow(t[0])

(3, 224, 224)